In [ ]:
import os, glob
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display

from google.colab import drive
drive.mount('/content/drive')

zip_path="/content/drive/MyDrive/L2_DATASET.zip"


#  unzip
extract_path="/content/voices_dataset"
os.makedirs(extract_path,exist_ok=True)
with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped to:",extract_path)
print("Files:",os.listdir(extract_path))

print("Extracted folder content:")
for r,ds,fs in os.walk(extract_path):
    print(r,"→",fs)

#  paths & files (reuse your unzip path) ---
extract_path="/content/voices_dataset"
audio_dir=os.path.join(extract_path,"L2_DATASET")
all_audio=sorted(glob.glob(os.path.join(audio_dir,"*.wav")))
print(f"Found {len(all_audio)} WAV files")
assert all_audio, "No WAV files found. Check /content/voices_dataset/L2_DATASET"

#  Helper: plot/save Log-Mel spectrogram for one file ----
def plot_mel(file_path, n_mels=64, fmax_hz=8000, hop_length=256, show=False, save_path=None):
    y, sr=librosa.load(file_path, sr=None, mono=True)
    S_mel=librosa.feature.melspectrogram(
        y=y, sr=sr, n_mels=n_mels, fmax=min(fmax_hz, sr/2), hop_length=hop_length
    )
    S_mel_db=librosa.power_to_db(S_mel, ref=np.max)

    plt.figure(figsize=(10,4))
    librosa.display.specshow(S_mel_db, x_axis='time', y_axis='mel',
                             sr=sr, hop_length=hop_length, fmax=min(fmax_hz, sr/2))
    plt.colorbar(label="dB")
    plt.title(f"Log-Mel Spectrogram — {os.path.basename(file_path)}")
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    elif show:
        plt.show()
    else:
        plt.close()

# 1) Show first file (and play it) ----
first_file=all_audio[0]
y, sr=librosa.load(first_file, sr=None, mono=True)
display(Audio(y, rate=sr))
plot_mel(first_file, show=True)

#  2) Loop over ALL files and save plots ----
out_dir=os.path.join(extract_path,"mel_plots")
os.makedirs(out_dir, exist_ok=True)

for fp in all_audio:
    png_name=os.path.splitext(os.path.basename(fp))[0]+"_mel.png"
    save_to=os.path.join(out_dir, png_name)
    plot_mel(fp, save_path=save_to)

print(f" Saved {len(all_audio)} Mel-spectrograms to: {out_dir}")
